# Examples with CSV files

> **These are Go notebooks**: In order to use the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

In [1]:
!*go mod edit -replace "github.com/umbralcalc/stochadex=/home/robert/Code/go/stochadex"

## To CSV

In [5]:
import (
	"os"

	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/inference"
)

%%

// Create a simulator.StateTimeStorage from a simulation run
storage := analysis.NewStateTimeStorageFromPartitions(
	// Instantiate the desired simulation state partitions
	[]*simulator.PartitionConfig{
		{
			Name:              "generated_data",
			Iteration:         &inference.DataGenerationIteration{
				Likelihood: &inference.NormalLikelihoodDistribution{},
			},
			Params:            simulator.NewParams(map[string][]float64{
				"mean": {1.8, 5.0, -7.3, 2.2},
                "covariance_matrix": {2.5, 0.0, 0.0, 0.0, 0.0, 9.0, 0.0, 0.0, 0.0, 0.0, 4.4, 0.0, 0.0, 0.0, 0.0, 17.0},
			}),
			InitStateValues:   []float64{1.3, 8.3, -4.9, 1.6},
			StateHistoryDepth: 1,
			Seed:              1234567,
		},
    },
	// Decide when should we stop the simulation
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 1000,
	},
	// Decide how time should evolve
	&simulator.ConstantTimestepFunction{
		Stepsize: 1.0,
	},
	// Input the initial time
	0.0,
)

// Retrieve a dataframe representing the data in one partition
df := analysis.GetDataFrameFromPartition(storage, "generated_data")

// Save the dataframe as a csv for later
file, _ := os.Create("data/test_csv.csv")
df.WriteCSV(file)

// Display the dataframe
fmt.Println(df)

[1001x5] DataFrame

    time     0         1        2          3
 0: 0.000000 1.300000  8.300000 -4.900000  1.600000
 1: 1.000000 0.551027  3.462065 -9.626819  4.480270
 2: 2.000000 2.678950  2.597829 -4.103315  1.770135
 3: 3.000000 -1.604998 5.943940 -6.395705  10.524925
 4: 4.000000 2.156654  2.037658 -5.657633  5.856814
 5: 5.000000 3.423471  3.824205 -9.942174  8.060977
 6: 6.000000 1.847624  6.755156 -11.975774 -2.426619
 7: 7.000000 0.754016  4.282124 -5.807639  0.886433
 8: 8.000000 -1.344118 8.621538 -8.682020  0.388972
 9: 9.000000 2.939750  3.391610 -8.781331  -0.650706
    ...      ...       ...      ...        ...
    <float>  <float>   <float>  <float>    <float>



## From CSV

In [7]:
import (
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Read in csv data directly into a simulator.StateTimeStorage
storage, _ := analysis.NewStateTimeStorageFromCsv(
	"data/test_csv.csv",
	0,
	map[string][]int{"generated_data" : {1, 2, 3, 4},},
	true,
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{IsTime: true}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{PartitionName: "generated_data"}}

// Create a scatter plot from partitions in a simulator.StateTimeStorage
scatter := analysis.NewScatterPlotFromPartition(storage, xRef, yRefs)

// Display the plot in a Go notebook
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")

## Likelihood estimation

In [16]:
import (
	"github.com/umbralcalc/stochadex/pkg/kernels"
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Read in csv data directly into a simulator.StateTimeStorage
storage, _ := analysis.NewStateTimeStorageFromCsv(
	"data/test_csv.csv",
	0,
	map[string][]int{"generated_data" : {1, 2, 3, 4},},
	true,
)

// Configure a partition for computing the exponentially-weighted rolling mean
meanPartition := analysis.NewVectorMeanPartition(
	analysis.AppliedAggregation{
		Name: "mean",
		Data: analysis.DataRef{PartitionName: "generated_data"},
		Kernel: &kernels.ExponentialIntegrationKernel{},
	}, 
	storage,
)

// Set the timescale for the exponential weighting
meanPartition.Params.Set("exponential_weighting_timescale", []float64{100.0})

// Run and add the mean partition to storage
storage = analysis.AddPartitionToStateTimeStorage(
	storage,
	meanPartition,
	map[string]int{"generated_data": 150},
)

// Reference the plotting data for the x-axis
xRef := analysis.DataRef{
	IsTime: true,
	TimeRange: &analysis.IndexRange{Lower: 150, Upper: 1000},
}

// Reference the plotting data for the y-axis
yRefs := []analysis.DataRef{{
	PartitionName: "mean",
	TimeRange: &analysis.IndexRange{Lower: 150, Upper: 1000},
}}

// Create a line plot from partitions in a simulator.StateTimeStorage
line := analysis.NewLinePlotFromPartition(storage, xRef, yRefs, nil)

// Display the plot in a Go notebook
gonb_echarts.Display(line, "width: 1024px; height:400px; background: white;")

## Online simulation inference

Line plots of params with confidences

## Offline simulation inference

Corner plot